In [11]:
!pip install langchain_community
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.9 MB/s eta 0:00:00


In [12]:
import os
os.environ['OPENAI_API_KEY'] = 'your api key'

In [13]:
import inspect
import re

from getpass import getpass
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, LLMMathChain, TransformChain, SequentialChain
from langchain_community.callbacks.manager import get_openai_callback

from langchain.llms import OpenAI

In [14]:
llm = OpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [15]:
llm_math = LLMMathChain(llm=llm)
llm_math( "What is area of a circle with radius 3.9?")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


{'question': 'What is area of a circle with radius 3.9?',
 'answer': 'Answer: 47.7594'}

In [16]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [17]:
llm_math = LLMMathChain(llm=llm, verbose=True)
count_tokens(llm_math, "What is 13 raised to the .3432 power?")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new LLMMathChain chain...
What is 13 raised to the .3432 power?```text
13**0.3432
```
...numexpr.evaluate("13**0.3432")...

Answer: 2.4116004626599237
> Finished chain.
Spent a total of 228 tokens


'Answer: 2.4116004626599237'

In [18]:
print(llm_math.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [19]:
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['question'], template='{question}')
llm_chain = LLMChain(prompt=prompt, llm=llm)

# we ask the llm for the answer with no context

count_tokens(llm_chain, "What is 13 raised to the .3432 power?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


Spent a total of 28 tokens


'\n\n13 raised to the .3432 power is approximately 2.999.'

In [20]:
print(inspect.getsource(llm_math._call))

    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        _run_manager = run_manager or CallbackManagerForChainRun.get_noop_manager()
        _run_manager.on_text(inputs[self.input_key])
        llm_output = self.llm_chain.predict(
            question=inputs[self.input_key],
            stop=["```output"],
            callbacks=_run_manager.get_child(),
        )
        return self._process_llm_result(llm_output, _run_manager)



In [21]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]

    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

In [22]:
clean_extra_spaces_chain = TransformChain(input_variables=["text"], output_variables=["output_text"], transform=transform_func)

In [23]:
clean_extra_spaces_chain.run('A random text  with   some irregular spacing.\n\n\n     Another one   here as well.')

'A random text with some irregular spacing.\n Another one here as well.'

In [24]:
template = """Paraphrase this text:

{output_text}

In the style of a {style}.

Paraphrase: """
prompt = PromptTemplate(input_variables=["style", "output_text"], template=template)

In [25]:
style_paraphrase_chain = LLMChain(llm=llm, prompt=prompt, output_key='final_output')

In [26]:
sequential_chain = SequentialChain(chains=[clean_extra_spaces_chain, style_paraphrase_chain], input_variables=['text', 'style'], output_variables=['final_output'])

In [27]:
input_text = """
deep learning is at the core of large                                  language models,
                  enabling them to understand and generate human language.                These models have achieved remarkable
                   success in various language-related tasks,
                    revolutionizing
the field of natural
                       language processing.
                       One of the key advantages of LLMs is their ability to perform transfer learning. They can be pre-trained on a large dataset and then fine-tuned for specific tasks.
        This
          reduces the need for large task-specific                      datasets and accelerates the development of natural                       language processing
          applications.
"""

In [28]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'a 90s rapper'})

Spent a total of 227 tokens


"Yo, deep learning be the heart of them big language models, makin' 'em able to comprehend and spit out human words. These models be killin' it in all sorts of language tasks, straight up changin' the game in natural language processing. One dope thing 'bout LLMs is they can do transfer learnin', gettin' pre-trained on a fat dataset and then tweaked for specific jobs. This means less need for huge task-specific data and faster development of language processing apps. Word."

In [29]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'formal'})

Spent a total of 213 tokens


'\nDeep learning serves as the foundation for extensive language models, empowering them to comprehend and produce human language. These models have demonstrated exceptional accomplishments in diverse language-oriented assignments, fundamentally transforming the realm of natural language processing. A significant benefit of LLMs is their capacity for transfer learning. They can be initially trained on a vast dataset and subsequently adjusted for particular tasks. This diminishes the necessity for extensive task-specific datasets and expedites the advancement of natural language processing applications.'

In [30]:
count_tokens(sequential_chain, {'text': input_text, 'style': 'academic'})

Spent a total of 219 tokens


'Deep learning serves as the foundation for expansive language models, empowering them to comprehend and produce human language. These models have demonstrated exceptional accomplishments in a multitude of language-related endeavors, fundamentally transforming the realm of natural language processing. A significant benefit of these large language models is their capacity for transfer learning, whereby they can be initially trained on a vast dataset and subsequently fine-tuned for specific tasks. This approach minimizes the necessity for extensive task-specific datasets and expedites the advancement of natural language processing applications.'